## 1. Importaciones y Configuración

In [ ]:
from typing import Any, Sequence, MutableSequence
from agent_framework import ContextProvider, Context, ChatAgent, ChatClientProtocol, ChatMessage, ChatOptions
from pydantic import BaseModel
import asyncio
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv
import os

load_dotenv()

print("✅ Importaciones completadas")

## 2. Modelo de Datos: UserInfo

Define la estructura de información que queremos recordar del usuario:

In [ ]:
class UserInfo(BaseModel):
    """Información del usuario que el agente recordará."""
    name: str | None = None
    age: int | None = None

print("✅ Modelo UserInfo definido")
print("📊 Campos: name (str), age (int)")

## 3. Implementación del Context Provider Personalizado

### UserInfoMemory: Proveedor de Memoria

Este Context Provider:
- **Almacena** información del usuario (nombre, edad)
- **Extrae** información automáticamente de los mensajes
- **Inyecta** contexto antes de cada interacción
- **Persiste** estado para serialización

### Métodos Principales:

1. **`invoking()`**: Se ejecuta ANTES de llamar al agente
   - Agrega instrucciones basadas en el estado actual
   - Modifica el contexto de la conversación

2. **`invoked()`**: Se ejecuta DESPUÉS de llamar al agente
   - Extrae nueva información de los mensajes
   - Actualiza la memoria

3. **`serialize()`**: Convierte el estado a JSON
   - Para persistencia en base de datos
   - Para transferencia entre sesiones

In [ ]:
class UserInfoMemory(ContextProvider):
    """Context Provider que mantiene memoria del usuario."""
    
    def __init__(self, chat_client: ChatClientProtocol, user_info: UserInfo | None = None, **kwargs: Any):
        """
        Crea el proveedor de memoria.
        
        Args:
            chat_client: Cliente de chat para extraer información
            user_info: Información inicial del usuario (opcional)
            **kwargs: Intentará crear UserInfo desde estos kwargs
        """
        self._chat_client = chat_client
        if user_info:
            self.user_info = user_info
        elif kwargs:
            self.user_info = UserInfo.model_validate(kwargs)
        else:
            self.user_info = UserInfo()

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Extrae información del usuario después de cada llamada al agente."""
        # Verificar si necesitamos extraer info de mensajes del usuario
        user_messages = [msg for msg in request_messages if hasattr(msg, "role") and msg.role.value == "user"]

        if (self.user_info.name is None or self.user_info.age is None) and user_messages:
            try:
                # Usar el cliente de chat para extraer información estructurada
                result = await self._chat_client.get_response(
                    messages=request_messages,
                    chat_options=ChatOptions(
                        instructions="Extrae el nombre y edad del usuario del mensaje si están presentes. Si no, retorna nulls.",
                        response_format=UserInfo,
                    ),
                )

                # Actualizar info del usuario con datos extraídos
                if result.value:
                    if self.user_info.name is None and result.value.name:
                        self.user_info.name = result.value.name
                    if self.user_info.age is None and result.value.age:
                        self.user_info.age = result.value.age

            except Exception:
                pass  # Falló la extracción, continuar sin actualizar

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs: Any) -> Context:
        """Proporciona contexto de información del usuario antes de cada llamada al agente."""
        instructions: list[str] = []

        if self.user_info.name is None:
            instructions.append(
                "Pregunta al usuario por su nombre y declina cortésmente responder preguntas hasta que lo proporcione."
            )
        else:
            instructions.append(f"El nombre del usuario es {self.user_info.name}.")

        if self.user_info.age is None:
            instructions.append(
                "Pregunta al usuario por su edad y declina cortésmente responder preguntas hasta que la proporcione."
            )
        else:
            instructions.append(f"La edad del usuario es {self.user_info.age}.")

        # Retornar contexto con instrucciones adicionales
        return Context(instructions=" ".join(instructions))

    def serialize(self) -> str:
        """Serializa la info del usuario para persistencia de thread."""
        return self.user_info.model_dump_json()

print("✅ UserInfoMemory implementado")
print("🧠 Capacidades:")
print("   - Extracción automática de nombre y edad")
print("   - Inyección de contexto dinámico")
print("   - Serialización para persistencia")

## 4. Creación y Ejecución del Agente con Memoria

In [ ]:
async def main():
    print("🚀 Iniciando agente con memoria...\n")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(
            async_credential=credential, 
            project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
            model_deployment_name=os.getenv("MODEL")
        )

        # Crear el proveedor de memoria
        memory_provider = UserInfoMemory(chat_client)

        # Crear el agente con memoria
        async with ChatAgent(
            chat_client=chat_client,
            instructions="Eres un asistente amigable. Siempre dirígete al usuario por su nombre.",
            context_providers=memory_provider,
        ) as agent:
            # Crear un nuevo thread para la conversación
            thread = agent.get_new_thread()

            print("💬 Conversación 1: Pregunta matemática (sin nombre ni edad)")
            print("="*60)
            response1 = await agent.run("Hola, ¿cuál es la raíz cuadrada de 9?", thread=thread)
            print(f"Agente: {response1.text}")
            print()

            print("💬 Conversación 2: Proporcionar nombre")
            print("="*60)
            response2 = await agent.run("Mi nombre es Ruaidhrí", thread=thread)
            print(f"Agente: {response2.text}")
            print()

            print("💬 Conversación 3: Proporcionar edad")
            print("="*60)
            response3 = await agent.run("Tengo 20 años", thread=thread)
            print(f"Agente: {response3.text}")
            print()

            # Acceder a la memoria a través del context_provider del thread
            user_info_memory = thread.context_provider.providers[0]
            if user_info_memory:
                print("\n" + "="*60)
                print("🧠 ESTADO DE LA MEMORIA")
                print("="*60)
                print(f"Nombre del Usuario: {user_info_memory.user_info.name}")
                print(f"Edad del Usuario: {user_info_memory.user_info.age}")
                print("="*60)

await main()

## 5. Demostración de Comportamiento Adaptativo

Veamos cómo el agente adapta sus respuestas basándose en la memoria:

In [ ]:
async def demo_adaptive_behavior():
    """Demuestra cómo el agente se adapta según la información recordada."""
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(
            async_credential=credential, 
            project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
            model_deployment_name=os.getenv("MODEL")
        )
        
        memory = UserInfoMemory(chat_client)
        
        async with ChatAgent(
            chat_client=chat_client,
            instructions="Eres un asistente que recomienda películas basándose en la edad del usuario.",
            context_providers=memory,
        ) as agent:
            thread = agent.get_new_thread()
            
            print("🎬 Demo: Recomendaciones de películas adaptativas\n")
            
            # Interacción 1: Sin edad
            print("1️⃣ Sin información de edad:")
            r1 = await agent.run("Recomiéndame una película", thread=thread)
            print(f"   {r1.text}\n")
            
            # Interacción 2: Proporcionar edad
            print("2️⃣ Usuario proporciona edad (8 años):")
            r2 = await agent.run("Tengo 8 años y mi nombre es Ana", thread=thread)
            print(f"   {r2.text}\n")
            
            # Interacción 3: Con edad conocida
            print("3️⃣ Con edad conocida:")
            r3 = await agent.run("Ahora sí, recomiéndame una película", thread=thread)
            print(f"   {r3.text}\n")

await demo_adaptive_behavior()

## 6. Extensión: Memoria Más Compleja

Ejemplo de un proveedor de memoria más sofisticado:

In [ ]:
from datetime import datetime

class EnhancedUserInfo(BaseModel):
    """Información extendida del usuario."""
    name: str | None = None
    age: int | None = None
    interests: list[str] = []
    preferences: dict[str, str] = {}
    last_interaction: datetime | None = None
    interaction_count: int = 0

class EnhancedMemory(ContextProvider):
    """Proveedor de memoria enriquecido con más capacidades."""
    
    def __init__(self, chat_client: ChatClientProtocol):
        self._chat_client = chat_client
        self.user_info = EnhancedUserInfo()
    
    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs: Any) -> Context:
        """Proporciona contexto enriquecido."""
        # Actualizar contador de interacciones
        self.user_info.interaction_count += 1
        self.user_info.last_interaction = datetime.now()
        
        instructions = []
        
        # Personalización basada en número de interacciones
        if self.user_info.interaction_count == 1:
            instructions.append("Este es el primer mensaje del usuario. Sé especialmente acogedor.")
        elif self.user_info.interaction_count > 10:
            instructions.append(f"Usuario frecuente ({self.user_info.interaction_count} interacciones).")
        
        # Información del usuario
        if self.user_info.name:
            instructions.append(f"Usuario: {self.user_info.name}")
        
        if self.user_info.interests:
            instructions.append(f"Intereses: {', '.join(self.user_info.interests)}")
        
        return Context(instructions=" ".join(instructions))
    
    def serialize(self) -> str:
        return self.user_info.model_dump_json()

print("✅ EnhancedMemory implementado")
print("🔍 Características adicionales:")
print("   - Seguimiento de intereses")
print("   - Preferencias del usuario")
print("   - Contador de interacciones")
print("   - Timestamp de última interacción")

## 7. Análisis y Conclusiones

### Ventajas de Context Providers Personalizados:

1. **Personalización Profunda**:
   - Adapta comportamiento según el usuario
   - Memoria persistente entre sesiones
   - Experiencia más natural

2. **Extracción Automática**:
   - No requiere formularios explícitos
   - Aprendizaje conversacional
   - Menos fricción para el usuario

3. **Flexibilidad**:
   - Puedes almacenar cualquier tipo de información
   - Lógica personalizada de inyección de contexto
   - Fácil extensión

4. **Separación de Responsabilidades**:
   - Memoria separada de la lógica del agente
   - Reutilizable entre diferentes agentes
   - Testing más sencillo

### Casos de Uso Prácticos:

1. **Asistentes Personales**:
   - Recordar preferencias del usuario
   - Adaptar tono y estilo
   - Sugerencias personalizadas

2. **E-commerce**:
   - Historial de compras
   - Preferencias de productos
   - Recomendaciones personalizadas

3. **Soporte al Cliente**:
   - Historial de tickets
   - Problemas recurrentes
   - Contexto de conversaciones previas

4. **Educación**:
   - Nivel de conocimiento
   - Estilo de aprendizaje
   - Progreso del estudiante

5. **Salud**:
   - Historial médico
   - Medicamentos
   - Preferencias de tratamiento

### Mejores Prácticas:

1. **Privacidad**: Manejar datos sensibles apropiadamente
2. **Consentimiento**: Informar al usuario sobre qué se recuerda
3. **Actualización**: Permitir correcciones de información
4. **Expiración**: Considerar TTL para datos antiguos
5. **Seguridad**: Encriptar información sensible

### Arquitectura de Producción:

```python
class ProductionMemory(ContextProvider):
    def __init__(self, user_id: str, db: Database):
        self.user_id = user_id
        self.db = db
        self.cache = {}
    
    async def load_from_db(self):
        """Cargar memoria desde base de datos."""
        data = await self.db.get_user_memory(self.user_id)
        self.cache = data
    
    async def save_to_db(self):
        """Guardar memoria en base de datos."""
        await self.db.update_user_memory(self.user_id, self.cache)
    
    async def invoked(self, ...):
        # Actualizar memoria
        await self.save_to_db()
```